# Importing Libraries and Cleaned Datasets

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels as sm
import scipy.stats as stats
from sklearn import linear_model, feature_selection, metrics, model_selection, impute, datasets
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

county_health_rank = pd.read_csv('county_rankings25.csv')
qol_22 = pd.read_csv('QOL.csv')
fips_codes = pd.read_csv('all_fips.csv')
county_vars = pd.read_csv('county_vars_2.csv')
race_groups = pd.read_csv('agg_race_variables.csv')

In [5]:
county_health_rank.shape

(3204, 796)

In [9]:
nulls = county_health_rank.isna().sum()

In [11]:
nulls.shape

(796,)

# Data Cleaning Plan
## County Health Rankings 2025
The Dataset 'County Health Rankings 2025' jas 3204 rows and 796 columms. The columns report socio economic metrics from all counties in the US. These metrics are unstacked with confidence intervals as columns adjacent to raw estimates. Some metrics report at a county level only, whilst others alos report racial group breakdows next to the totals. Though it is a comprehensive dataset, like most of its kind, there is much missing data, especially for minority groups, with communities like Native Hawaiian or Pacific Islander not appearing at all in a majority of counties. 

The data wrangling process will involve using a funciton to select the target variables, renaming, and normalizing data types. I will, then, validate the resulting dataframe, and applying the standardized hierarchical index (county, state).

Since dataset contains hundreds of variables, I created a small dataframe of our variables of interest and normalized names to facilate creating the subset itself and its naming conventions. 

Note: much of the numeric data is in percentages, separate subsets will include totals and percentages and will be used depnding on type of analysis and indexing needed.
## Quality of Life Data 2022
Since this study began with this study and happened to contain data for the US 2020 General Election at a county level, I will subset that category and merge with the table above. 
## FIPS & States ID table
I will merge both tables above to a previosly created 'all_fips' (which serves as a sort of  'fact table'-listing county names, state names, state abbreviations, and corresponding fips codes) by index. 

I will validate the data and clean any merging errors like the creation of duplicate columns and/or changes to naming conventions (such as adding suffixes/prefixes to duplicate and original colums).

Finally, I will merge both of these new tables by the standardized index mentioned above.

## Groupings and Subsets
Depending on the structure of the final merge, I may use smaller subsets and/or different data structures and groupings for specialized analysis and testing .

# Data Cleaning

## County Rankings tables

### Make Subset Function

In [12]:
# defining variable for function to select target vars, dictionary to rename, and set 'fips' as 'str' dtype
county_health_subset = county_vars['raw_variable'].tolist()
county_name_list = county_vars[' new_name'].tolist()
county_new_names = dict(zip(county_health_subset, county_name_list))

In [13]:
def make_subset(df, target_vars, target_names):
    df = df[target_vars]
    df = df.rename(columns=target_names) #dictionary defined from target variables & names list
    df['fips'] = df['fips'].astype('str')
    return df

In [14]:
counties = make_subset(county_health_rank, county_health_subset, county_new_names)

### Clean colums function

In [ ]:
# list of state names from master fips to drop from county namers column
state_names = fips_codes['state'].unique()
state_names = state_names.tolist()
state_names.append('United States')
# variable name referring to state 
def clean_subset(df):
    df = df[~df['county'].isin(state_names)]
    df = df.round(3)
    return df

In [ ]:
counties = clean_subset(counties)

## QOL- Political Party 2020

In [ ]:
qol_target_vars = ['2020PopulrVoteParty', 'NMCNTY', 'FIPS', 'LSTATE']
qol_new_names = {'2020PopulrVoteParty': 'political_party', 'NMCNTY':'county', 'FIPS':'fips', 'LSTATE':'state_abbr'}

In [ ]:
pol_party_2020 = make_subset(qol_22, qol_target_vars, qol_new_names)

## subsets and groupings for drill-down analysis 

## Variable-based subsets

In [ ]:
inequity_vars = ['state_abbr','county','fips', 'income_inequality', 'gender_pay_gap', 'school_segration_index', 'residential_segregation','school_funding_gap']
# Socio-economic indicators with racial breakdown
racial_category_subset = race_groups['raw_variable'].tolist()
racial_category_names = race_groups['new_name'].tolist()
racial_category_cols = dict(zip(racial_category_subset, racial_category_names))

In [ ]:
# cleaning and reshaping dataframe with variable containing racial-breakdowns
race_subset = make_subset(county_health_rank, racial_category_subset, racial_category_cols)
counties_race = clean_subset(race_subset)
# reshape dataframe
# Melt the dataframe to long format
counties_race = counties_race.melt(
    id_vars=['state_abbr', 'county', 'fips', 'county_population'], 
    var_name='metric_race', 
    value_name='value'
)

# Split the combined 'metric_ethnicity' column
counties_race[['metric', 'race']] = counties_race['metric_race'].str.extract(r'(.+?)-(.+)')

# Drop the original combined column
counties_race = counties_race.drop(columns='metric_race')

# Pivot to have metrics as columns
counties_race = counties_race.pivot_table(
    index=['state_abbr', 'county', 'fips', 'race', 'county_population'],
    columns='metric',
    values='value'
).reset_index()

# Clean up column names if needed
counties_race.columns.name = None

### County-Level socio-economic outcomes (no racial breakdown)

In [ ]:
aggregate_outcomes = counties.drop(columns=['child_mortality', 'child_poverty_percentage', 'firearm_fatalities',
       'life_expectancy', 'median_income', 'premature_death'])
agg_vars = aggregate_outcomes.columns.to_list()
agg_vars.append('political_party')

## Total Values (vs percentages)

In [ ]:
pop_vars = ['state_abbr','county','fips', 'population']
county_pop = counties[pop_vars]
# find columns w percentage ouputs by assigining target string to variable ans using lamda  func to filter col names and assign to list
percent_str = 'percentage'
percent_vars = list(filter(lambda x: percent_str in x, county_name_list))
counties_totals = counties.copy()
counties_pop_col = 'population'

#### totals function

In [ ]:
# multiplies each percentage column by county polulation to yield total amounts
def calc_total_pop(df, cols, pop_col):
    for i in df[cols]:
        df[i] = df[i] * df[pop_col]
    df.columns = df.columns.str.replace('_percentage', '')
    return df

In [ ]:
counties_totals= calc_total_pop(counties_totals, percent_vars, counties_pop_col)

### Totals- broken down by race

In [ ]:
race_pop_col = 'county_population'
race_percentage_vars = [ 'population_percentage', 'child_poverty_percentage']
racial_breakdown_total = counties_race.copy()
counties_race_totals = calc_total_pop(racial_breakdown_total, race_percentage_vars, race_pop_col)

Note: Many missing values

## Merge for Analysis

In [ ]:
fips_codes['fips'] = fips_codes['fips'].astype('str')
index = ['state_abbr', 'county', 'fips']
fips_codes.reset_index(inplace=True)
pol_party_2020.reset_index(inplace=True) #state_abbr
counties.reset_index(inplace=True) #state_abbr
# setting common indices

### Merge dataframes using 'state' with 'fact' table ('all_fips') and set indices for uniform id variables

In [ ]:
fips_codes.set_index(index)
pol_party_2020.set_index(index)
counties.set_index(index)
pol_party_indexed = pd.merge(pol_party_2020, fips_codes, on=index, how='inner')
counties_indexed = pd.merge(counties, fips_codes, on=index, how='inner')

In [ ]:
pol_party_indexed= pol_party_indexed.drop(columns='state')
counties_indexed= counties_indexed.drop(columns='state')

### Merge counties and pol_party df's on common multi-index

In [ ]:
# function to clean merged tables
def clean_merged(df):
    # Drop duplicate rows in column: 'fips'
    df = df.drop_duplicates(subset='fips')
    df = df[~df['county'].isin(state_names)]
    df = df.round(3)
    return df

In [ ]:
counties_pol_party = clean_merged(counties_pol_party)
counties_pol_party.reset_index()

## Create combined dataframe with county-level variables and variables broken down by race 

Subset merged df with variables for aggregate data (not contained in race dataframe) to avoid duplicate columns

In [ ]:
test_agg = counties_pol_party[agg_vars]
all_sets= counties_race.merge(test_agg, how='left', on=index)
all_sets.set_index(['state_abbr', 'county', 'fips','race'], inplace=True)

In [ ]:
all_sets.reset_index(inplace=True)
all_sets['race'] = all_sets['race'].astype('category')

In [ ]:
all_sets.reset_index()

## Address Missing Data 

### Aggregate data

In [ ]:
missing = counties_pol_party.isnull().sum()
print(missing)

In [ ]:
counties_pol_party.reset_index(inplace=True)
counties_pol_party = counties_pol_party.drop(columns='index')

#### Use KNN Imputation to address missing data

In [ ]:
non_numeric_cols = ['state_abbr', 'county', 'fips', 'political_party']
counties_numeric_cols = ['hs_diploma_percentage',
       'unemployed_percentage', 'population', 'median_income',
       'school_segration_index', 'gender_pay_gap', 'uninsured__percentage',
       'school_funding_gap', 'residential_segregation', 'premature_death',
       'income_inequality', 'child_poverty_percentage', 'life_expectancy',
       'child_mortality', 'high_housing_cost_percentage', 'firearm_fatalities',
       'child_care_cost_burden']

In [ ]:
imputer = KNNImputer(n_neighbors=2)
# running inputation for numeric columns only
countypol_sets_numeric = pd.DataFrame(imputer.fit_transform(counties_pol_party[counties_numeric_cols]), columns=counties_numeric_cols)
# concatenating to non0numeric columns to recreate entire df
countypol_filled = pd.concat([counties_pol_party[non_numeric_cols].reset_index(drop=True), countypol_sets_numeric], axis=1)

### hierachical dataframe- brokendown by race
There is a great deal of missing data for racial categories, handling missing data will differ from above

In [ ]:
all_sets_missing = all_sets.isnull().sum()

In [ ]:
# drop missing population values- indicates no county reporting for racial group
# too much missing data for imputation to entire seet- median income has highest reporting for racial groups
all_sets = all_sets.dropna(subset=['population','median_income'])
# dropping colums w more than half of records missing
#all_sets = all_sets.drop(columns=['firearm_fatalities', 'child_mortality'])
all_sets.reset_index()

#### see categorical descriptions

In [ ]:
print(all_sets.groupby('race').describe())

- nhopi missig summary stats across all metrics- will drop this category 

In [ ]:
all_sets['race'] = all_sets['race'].cat.remove_categories('nhopi')

#### Data Imputation
after removing mostly incomplete columns and categories, same KNN imputation will be used 

In [ ]:
numeric_cols = ['county_population',
       'child_mortality', 'child_poverty_percentage', 'firearm_fatalities',
       'life_expectancy', 'median_income', 'population_percentage',
       'premature_death', 'hs_diploma_percentage', 'unemployed_percentage',
       'population', 'school_segration_index', 'gender_pay_gap',
       'uninsured__percentage', 'school_funding_gap',
       'residential_segregation', 'income_inequality',
       'high_housing_cost_percentage', 'child_care_cost_burden']

imputer = KNNImputer(n_neighbors=2)
all_sets_numeric = pd.DataFrame(imputer.fit_transform(all_sets[numeric_cols]), columns= numeric_cols)
all_sets_filled = pd.concat([all_sets[non_numeric_cols].reset_index(drop=True), all_sets_numeric], axis=1)

In [ ]:
all_sets_filled.head()

### Standardizing Values

In [ ]:
independent_vars = ['income_inequality', 'gender_pay_gap',
       'school_segration_index', 'residential_segregation',
       'school_funding_gap']

dependent_vars = ['hs_diploma_percentage', 'unemployed_percentage', 'population', 'median_income', 'uninsured__percentage', 'premature_death', 'child_poverty_percentage', 'life_expectancy', 'child_mortality', 'high_housing_cost_percentage', 'firearm_fatalities', 'child_care_cost_burden']
vars_to_z = independent_vars + dependent_vars
#standardize_frame = counties[vars_to_z]
standardized_frame = counties.dropna()
standardized_all_filled = all_sets_filled.copy()

In [ ]:
# multiplies each percentage column by county polulation to yield total amounts
def calc_z(df, cols):
    for i in df[cols]:
        df[i] = np.abs(stats.zscore(df[i])) 
    return df

In [ ]:
z_counties= calc_z(countypol_filled, counties_numeric_cols)
z_all_sets_filled= calc_z(all_sets_filled, numeric_cols)

# Insights

## Mutual Information: finding highly dependent Variables

### County-Level

#### Creating a Matrix of pairs fo each dependent var w independent var

Attempted model with KNN inputation and results were drastically different, thus rejected inputed values as overwhlmingly affecting dta integrity. therefore attempting original data with dropped missing values

In [ ]:
counties_pol_raw = counties_pol_party.dropna()

In [ ]:
# Initialize a matrix to store mutual information
mutual_info_matrix = np.zeros((len(independent_vars), len(dependent_vars)))

# Calculate mutual information for each independent-dependent variable pair
for i, var1 in enumerate(independent_vars):
    for j, var2 in enumerate(dependent_vars):
        X = counties_pol_raw[[var1]]
        y = counties_pol_raw[var2]
        mutual_info = mutual_info_regression(X, y)
        mutual_info_matrix[i, j] = mutual_info[0]  # Store the mutual info scalar

# Convert the matrix to a DataFrame for easier viewing
mutual_info_df = pd.DataFrame(mutual_info_matrix, index=independent_vars, columns=dependent_vars)

# Display the mutual information DataFrame
print(mutual_info_df)

#### Visualizing Pairwise Mutual Information as Heatmap

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(mutual_info_df, annot=True, cmap='flare', square=True)
plt.title('Pairwise Mutual Information- Inequity')
plt.show()

#### Getting top values of dependent pairs

In [ ]:
top_var_pairs = mutual_info_df.unstack().sort_values(ascending=False).head(12)
print(top_var_pairs)

### Selecting most 'infromative' variables
- x: 'school_funding_gap', 'income_inequality'
- y: 'child_poverty_percentage', 'firearm_fatalities', 'median_income' , 'life_expectancy', 'premature_death'

In [ ]:
high_mutual_info_vars = ['state_abbr', 'county', 'fips', 'school_funding_gap', 'child_poverty_percentage', 'income_inequality', 'firearm_fatalities', 'median_income' , 'life_expectancy', 'premature_death', 'political_party']
agg_select_subset = counties_pol_raw[high_mutual_info_vars]
x_vars= ['school_funding_gap', 'income_inequality'],
y_vars= ['child_poverty_percentage', 'firearm_fatalities', 'median_income' , 'life_expectancy', 'premature_death']

## Correlation and Covariance

In [ ]:
corr_matrix = agg_select_subset.corr(method='kendall', numeric_only=True).round(2)
sns.heatmap(corr_matrix, annot=True, cmap='flare')

### Correlations between selected explainer and selected outcome variables

In [ ]:
print(agg_select_subset[y_vars].corrwith(agg_select_subset['school_funding_gap']))

In [ ]:
print(agg_select_subset[y_vars].corrwith(agg_select_subset['income_inequality']))

In [ ]:
sns.pairplot(agg_select_subset, 
    x_vars= ['school_funding_gap', 'income_inequality'],
    y_vars= ['child_poverty_percentage', 'firearm_fatalities', 'median_income' , 'life_expectancy', 'premature_death'],
    )     

## Distributions

In [ ]:
agg_select_subset.hist(figsize=(12,12), layout=(3,3), color='teal', ec='black', sharex=False);

In [ ]:
sns.displot(agg_select_subset['school_funding_gap'])

In [ ]:
sns.displot(agg_select_subset['income_inequality'])

**Note**: not normally distributed

## Variation in each variable

In [ ]:
ax = agg_select_subset[high_mutual_info_vars].plot(figsize=(20,15))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5));

## Drill down on race

In [ ]:
all_sets_raw = all_sets.dropna()
all_sets_raw.reset_index(inplace=True)

In [ ]:
#high_mutual_info_vars.remove('race')
race_high_val_mutual_info_vars = high_mutual_info_vars.copy()
race_high_val_mutual_info_vars.append('race')
race_select_subset = all_sets_raw[race_high_val_mutual_info_vars]

### Pair Plots

In [ ]:
sns.barplot(race_select_subset, x='race', y='premature_death')

In [ ]:
race_select_subset.groupby('race')['child_poverty_percentage'].median()

In [ ]:
sns.FacetGrid(data = race_select_subset, col = 'race').map(plt.scatter, 'school_funding_gap', 'child_poverty_percentage').add_legend()
plt.show()

In [ ]:
race_select_subset.columns

In [ ]:
sns.set_theme()

g = sns.lmplot(
    data=race_select_subset,
    x='school_funding_gap', y='firearm_fatalities', hue='race',
    height=5
)

g.set_axis_labels('school funding', 'firearm fatalities')

In [ ]:
sns.set_theme()

g = sns.lmplot(
    data=race_select_subset,
    x='school_funding_gap', y='child_poverty_percentage', hue='race',
    height=5
)

g.set_axis_labels('school funding', 'child poverty')

## Comparing distributions between inequality metric quantiles

In [ ]:
q1_school_funding = agg_select_subset['school_funding_gap'].quantile(.25)
q2_school_funding = agg_select_subset['school_funding_gap'].quantile(.5)
q3_school_funding = agg_select_subset['school_funding_gap'].quantile(.75)

In [ ]:
q1_income_inequality = agg_select_subset.groupby('county')['income_inequality'].quantile(.25)
q2_income_inequality = agg_select_subset.groupby('county')['income_inequality'].quantile(.5)
q3_income_inequality = agg_select_subset.groupby('county')['income_inequality'].quantile(.75)

## Political Party

In [ ]:
agg_select_subset.columns

## Regression

In [ ]:
X = agg_select_subset[['school_funding_gap', 'income_inequality']]
y = agg_select_subset[['premature_death']]
X_train, X_test,\
  y_train, y_test = train_test_split(X, y,
                    test_size=0.4,
                    random_state=1)
reg = linear_model.LinearRegression()
model1 = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(projection='3d')

ax.scatter(X_test['school_funding_gap'], X_test['income_inequality'],
           y_test, color='coral', alpha=0.7, label='Premature Death')

x1_range = np.linspace(X_test['school_funding_gap'].min(), X_test['school_funding_gap'].max(), 100)
x2_range = np.linspace(X_test['income_inequality'].min(), X_test['income_inequality'].max(), 100)
x1, x2 = np.meshgrid(x1_range, x2_range)

z = model.predict(np.c_[x1.ravel(), x2.ravel()]).reshape(x1.shape)

ax.plot_surface(x1, x2, z, color='aquamarine', alpha=0.4, rstride=100, cstride=100)

ax.set_xlabel('School Funding')
ax.set_ylabel('Income Inequality')
ax.set_zlabel('Premature Death')
ax.set_title('Multiple Linear Regression Best Fit Line (3D)')

plt.show()

In [ ]:
# regression coefficients
print('Coefficients: ', reg.coef_)

# variance score: 1 means perfect prediction
print('Variance score: {}'.format(reg.score(X_test, y_test)))

### Residual Errors

In [ ]:
# plot for residual error

# setting plot style
plt.style.use('fivethirtyeight')

# plotting residual errors in training data
plt.scatter(reg.predict(X_train),
            reg.predict(X_train) - y_train,
            color="orange", s=10,
            label='Train data')

# plotting residual errors in test data
plt.scatter(reg.predict(X_test),
            reg.predict(X_test) - y_test,
            color="teal", s=10,
            label='Test data')

# plotting line for zero residual error
plt.hlines(y=0, xmin=0, xmax=50, linewidth=2)

# plotting legend
plt.legend(loc='upper left')

# plot title
plt.title("Residual errors")

# method call for showing the plot
plt.show()

In [ ]:
#regr = linear_model.LinearRegression()  
#model1= regr.fit(X, Y)

In [ ]:
#r2 = regr.score(X,Y)
#coefficients =regr.coef_
#intercept = regr.intercept_